<a href="https://colab.research.google.com/github/devrajPriyadarshi/Automatic-Tagging-of-3D-Assets/blob/main/Catagorise3D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Uncomment when running in Google Colab
# !pip install trimesh
import os
import trimesh
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from matplotlib import pyplot as plt
from keras.models import load_model
from trimesh import transformations

In [ ]:
class OrthogonalRegularizer(keras.regularizers.Regularizer):
    def __init__(self, num_features, l2reg=0.001):
        self.num_features = num_features
        self.l2reg = l2reg
        self.eye = tf.eye(num_features)

    def __call__(self, x):
        x = tf.reshape(x, (-1, self.num_features, self.num_features))
        xxt = tf.tensordot(x, x, axes=(2, 2))
        xxt = tf.reshape(xxt, (-1, self.num_features, self.num_features))
        return tf.reduce_sum(self.l2reg * tf.square(xxt - self.eye))

    def get_config(self):
        return {  'num_features': self.num_features,
                  'l2reg': self.l2reg
                }


In [ ]:
from keras.models import load_model

# When running as a python script 
model = load_model("PointNet_model.tf", custom_objects={"OrthogonalRegularizer":OrthogonalRegularizer})

# When running in Google Colab
# model = load_model("/content/PointNet_model.tf", custom_objects={"OrthogonalRegularizer":OrthogonalRegularizer})
model.summary()

"""
THE CURRENT MODEL IS A POINTNET NEURAL NETWORK, PRE-TRAINED ON THE ModelNet10 DATASET.
LINK TO THE IMPLEMENTATION: https://www.kaggle.com/code/devrajpriyadarshi/assesttag
"""

In [ ]:
CLASS_MAP = {0: 'monitor',
 1: 'bathtub',
 2: 'table',
 3: 'dresser',
 4: 'night_stand',
 5: 'sofa',
 6: 'bed',
 7: 'toilet',
 8: 'chair',
 9: 'desk'}

print("CLASS MAP:", CLASS_MAP)

In [ ]:
"""
SINCE ASSESTS DOWNLOADED FROM NET WILL HAVE RANDOM ORIENTATION, THIS WAS INCLUDED TO 
WORK AS A ORIENTATION INVARIANT CLASSIFIER!

THE INACCURARY DUE TO RANDOM ORIENTATION IS DUE TO THE LIMITATIONS OF THE 
NEURAL NETWORK ARCHITECTURE.

THIS CAN BE FIXED BY IMPLEMENTING THE:
"Rotation Invariant Convolutions for 3D Point Clouds Deep Learning"
LINK : https://github.com/hkust-vgd/riconv
"""

"""
PI = 3.14159265359
rotX = transformations.rotation_matrix(PI/2, [1,0,0])
rotY = transformations.rotation_matrix(PI/2, [0,1,0])
rotZ = transformations.rotation_matrix(PI/2, [0,0,1])

ROT = ["I", "X", "Y" ,"XX" ,"XY", "YX", "YY", "XXX", "XXY", "XYX", "XYY", "YXX", "YYX", "YYY", "XXXY", "XXYX", "XXYY", "XYXX", "XYYY", "YXXX", "YYYX", "XXXYX", "XYXXX", "XYYYX"]

def predictModel(file_path):
    max_score = 0
    classId = 0
    for r in ROT:
        mesh = trimesh.load(file_path)
        if r == "I":
            points = mesh.sample(2048)
            points = points[None ,:]
            pred = model.predict(points)
            score = max(max(pred))
            print(CLASS_MAP[tf.math.argmax(pred, -1).numpy()[0]], score)
            if score > max_score:
                max_score = score
                classID = tf.math.argmax(pred, -1).numpy()[0]
        else:
            for x in r:
                if x == "X":
                    mesh.apply_transform(rotX)
                elif x == "Y":
                    mesh.apply_transform(rotY)
            points = mesh.sample(2048)
            points = points[None ,:]
            pred = model.predict(points)
            score = max(max(pred))
            print(CLASS_MAP[tf.math.argmax(pred, -1).numpy()[0]], score)
            if score > max_score:
                max_score = score
                classID = tf.math.argmax(pred, -1).numpy()[0]
                
    return CLASS_MAP[classID], max_score

"""
            

In [ ]:
def predictModelRotationVariant(file_path):

    mesh = trimesh.load(file_path, force="mesh")
    trimesh.exchange.off.export_off(mesh)
    points = mesh.sample(2048)
    points = points[None, :]
    pred = model.predict(points)
    score = max(max(pred))
    classID = tf.math.argmax(pred, -1).numpy()[0]
            
    return CLASS_MAP[classID], score

In [ ]:
# When running as a python script 
fp3 = "Models/Model4.ply"

# When running in Google Colab
# fp3 = "/content/Models/Model4.ply"

Class, Score = predictModelRotationVariant(fp3)
print("\nFinal :", Class, Score)

mesh = trimesh.load(fp3)
mesh.show()

In [ ]:
# When running as a python script 
folder = "Models/"

# When running in Google Colab
# folder = "/content/Models/"

data = []
for count, filename in enumerate(os.listdir(folder)):
    Class, Score = predictModelRotationVariant(folder + filename)
    data.append([filename, Class])
    
Assets = pd.DataFrame(data, columns=['Model', 'Class'])
Assets

In [ ]:
Assets.to_csv("3D-Assets.csv")